<a href="https://colab.research.google.com/github/gabisouzaes/AnalisadorReunioesDevComGemini/blob/main/AnalisadorReunioesDevComGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
!pip install -q google-generativeai
!pip install -q google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [125]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [126]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [127]:
# Importa as bibliotecas

# --- Importações Padrão do Python ---
import os  # Para interagir com o sistema operacional (ex: variáveis de ambiente)
import json  # Para trabalhar com dados no formato JSON

# --- Importações para Interação com APIs do Google (incluindo Google Docs) ---
# Biblioteca principal para construir serviços para APIs do Google
from googleapiclient.discovery import build

# Bibliotecas para autenticação no Google Cloud e Colab
from google.colab import auth, userdata  # 'auth' para autenticar o usuário no Colab, 'userdata' para acessar Secrets
import google.auth # Para obter as credenciais padrão após a autenticação

# --- Importações para o Google Generative AI (Gemini) ---
import google.generativeai as genai_sdk # SDK Padrão do Google Gemini, nomeado como 'genai_sdk'

# --- Importações para Exibição no Colab (Opcional, mas útil) ---
from IPython.display import Markdown, display # Para formatar e exibir saídas de forma mais rica

# --- Importação para Datas (Opcional, mas útil para registrar quando as análises são feitas) ---
from datetime import datetime

print("Todas as bibliotecas e funções necessárias foram importadas.")
print("Lembre-se de configurar sua API Key do Gemini e autenticar o usuário para o Google Docs.")

Todas as bibliotecas e funções necessárias foram importadas.
Lembre-se de configurar sua API Key do Gemini e autenticar o usuário para o Google Docs.


In [128]:
TECH_LEAD_NAME = "Gabriella"
NOME_MODELO_GEMINI_STRING = "gemini-2.0-flash"

In [129]:
# Inicializa o serviço do Google Docs
try:
    auth.authenticate_user()
    print("Usuário autenticado para serviços Google.")
    creds, project = google.auth.default()
    # Inicializar o serviço do Google Docs
    docs_service = build('docs', 'v1', credentials=creds)
    print("Serviço do Google Docs pronto para uso.")
except Exception as e:
    print(f"Erro na autenticação do Google: {e}")

Usuário autenticado para serviços Google.
Serviço do Google Docs pronto para uso.


In [130]:
PROMPT_INSIGHTS = """Você é uma assistente de IA com especialidade em identificar tendências de desenvolvimento em séries de conversas 1:1.
As reuniões analisadas são entre {nome_da_lider} (líder técnica) e um de seus liderados.
Interprete o conjunto de anotações de reuniões fornecido abaixo como um todo, refletindo a jornada de desenvolvimento deste liderado específico com {nome_da_lider}.

Anotações das Reuniões Agrupadas:
---
{textos_combinados_das_reunioes}
---

Com base em todas as anotações fornecidas, elabore um resumo com suas percepções gerais sobre os pontos de desenvolvimento do liderado. Em seu resumo, procure destacar:

* As principais áreas de foco para desenvolvimento (técnicas e comportamentais) que se mantiveram ao longo do tempo.
* Evidências claras de progresso ou, alternativamente, áreas onde o desenvolvimento pareceu estagnado ou enfrentou dificuldades.
* Desafios ou obstáculos que foram mencionados ou observados de forma recorrente.
* De 1 a 3 recomendações principais ou próximos passos para o desenvolvimento futuro do liderado.

Apresente suas percepções de forma clara, direta e concisa.
"""
def gerar_insights_de_documentos_agrupados(
    lista_ids,
    servico_docs,
    nome_lider,
    template_prompt,
    modelo_gemini_obj
):
    """Lê, combina e envia conteúdos para análise do Gemini (versão simplificada)."""
    textos_docs = []
    for doc_id in lista_ids:
        texto_lido = ler_transcricao_do_google_docs(doc_id, servico_docs)
        if texto_lido != "DOCUMENTO_VAZIO" and "ERRO" not in texto_lido:
            textos_docs.append(f"--- CONTEÚDO DOCUMENTO (ID: {doc_id}) ---\n{texto_lido}\n--- FIM DOCUMENTO (ID: {doc_id}) ---\n\n")

    textos_combinados = "".join(textos_docs)
    if not textos_combinados.strip():
        return "ERRO: Nenhum conteúdo de documento para analisar."

    prompt_final = template_prompt.format(
        nome_da_lider=nome_lider,
        textos_combinados_das_reunioes=textos_combinados
    )
    # Chamada direta à API, sem try-except para máxima simplificação aqui.
    # Erros da API irão parar o script.
    resposta = modelo_gemini_obj.generate_content(prompt_final)
    return resposta.text

# --- 6. EXECUÇÃO PRINCIPAL DA ANÁLISE ---
print("\n--- Execução Principal da Análise ---")

if docs_service and modelo_gemini_objeto:
    lista_de_document_ids = obter_lista_de_ids_google_docs()

    if lista_de_document_ids:
        print("\n   Iniciando geração de insights...")
        insights_finais = gerar_insights_de_documentos_agrupados(
            lista_de_document_ids,
            docs_service,
            TECH_LEAD_NAME,
            PROMPT_INSIGHTS,
            modelo_gemini_objeto
        )
        print("\n   --- RESULTADO DA ANÁLISE DA IA ---")
        display(Markdown(str(insights_finais)))
    else:
        print("\n   Nenhum ID de documento fornecido. Análise não realizada.")
else:
    print("\n   ERRO CRÍTICO: 'docs_service' ou 'modelo_gemini_objeto' não inicializado(s). Verifique a Seção 4.")

print("\n--- Fim do Script ---")



--- Execução Principal da Análise ---

   --- Coleta de IDs de Documentos do Google Docs ---
   Insira o ID do Documento nº 1 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 1UMd1lqlvB_VWfW442obW0yjD3tlMt_7U55sxntLA3ho
   ID '1UMd1lqlvB_VWfW442obW0yjD3tlMt_7U55sxntLA3ho' adicionado à lista.
   Insira o ID do Documento nº 2 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 1_SH0o76SJD9_htSe8S5WegYR08sjzc89fPr0uhp1bjo
   ID '1_SH0o76SJD9_htSe8S5WegYR08sjzc89fPr0uhp1bjo' adicionado à lista.
   Insira o ID do Documento nº 3 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 1HVlICG87FCSLyRxqrZM84Umx7YoqgiV-lV6EC2CQfLA
   ID '1HVlICG87FCSLyRxqrZM84Umx7YoqgiV-lV6EC2CQfLA' adicionado à lista.
   Insira o ID do Documento nº 4 (ou digite 'fim' para concluir, ou pressione Enter para concluir se já adicionou algum): 1i7lcXMHI4LT9DZuUQ5_sEOMrFeC8vCi6GTrWUdMFUP8
   ID '1i7lcXMHI4LT

## Percepções sobre o Desenvolvimento de Túlio:

As reuniões 1:1 entre Gabriella e Túlio revelam uma jornada de desenvolvimento focada em duas áreas principais: **gestão da ambiguidade e comunicação do impacto do trabalho**. Inicialmente, o foco era em estruturar o pensamento analítico, organizar tarefas e otimizar a produtividade. Rapidamente, a atenção se deslocou para a dificuldade de Túlio em lidar com a incerteza, sua tendência a minimizar o próprio valor e a importância de comunicar o impacto de suas entregas de forma mais eficaz.

**Áreas de Foco Constantes:**

*   **Gestão da Ambiguidade:** A dificuldade em lidar com a incerteza e manter uma atitude construtiva em processos de transformação é um tema recorrente. Túlio demonstra tendência a se cobrar demais por não ter todas as respostas e a se sentir paralisado diante do desconhecido.
*   **Comunicação e Auto-valorização:** Túlio tende a minimizar o valor de suas entregas e não comunicar seu impacto de forma clara. Isso afeta a percepção que os outros têm do seu trabalho e pode limitar seu crescimento.

**Evidências de Progresso e Dificuldades:**

*   **Progresso:** Túlio demonstra agilidade na execução de tarefas (desdobramento de metas), busca ativamente conhecimento através de trocas com outros BAs e conexões com o time de CS, e tem sido fundamental para destravar pontos em projetos importantes. Ele também demonstra abertura para receber feedback e trabalhar em seus pontos fracos.
*   **Dificuldades:** A principal dificuldade reside na persistência da ansiedade e nervosismo em situações de incerteza, o que pode prejudicar a fluidez do trabalho. A autocrítica excessiva e a tendência a minimizar o próprio valor também continuam sendo desafios.

**Desafios e Obstáculos Recorrentes:**

*   **Autocrítica e Perfeccionismo:** Túlio se cobra demais por não ter todas as respostas, o que gera ansiedade e pode levá-lo a descredibilizar o processo.
*   **Dificuldade em Lidar com a Incerteza:** A falta de clareza em processos de transformação o deixa desconfortável e pode prejudicar sua capacidade de manter uma atitude construtiva.
*   **Comunicação do Impacto:** Túlio precisa aprender a comunicar o valor de suas entregas de forma mais eficaz, destacando o impacto e a complexidade das tarefas.

**Recomendações e Próximos Passos:**

1.  **Focar na mentalidade de crescimento:** Encorajar Túlio a ver a incerteza como uma oportunidade de aprendizado e desenvolvimento. Trabalhar na aceitação de que nem sempre terá todas as respostas e que o esforço investigativo é mais importante que a perfeição.
2.  **Desenvolver habilidades de comunicação de impacto:** Continuar incentivando Túlio a reconhecer o valor de suas entregas e comunicar seu impacto de forma clara e concisa, utilizando dados e exemplos concretos para demonstrar seus resultados.
3.  **Continuar o acompanhamento da saúde mental:** Manter o espaço aberto para conversas sobre temas subjetivos, como saúde mental e bem-estar. Auxiliar Túlio a encontrar estratégias para lidar com a ansiedade e o estresse no ambiente de trabalho, buscando apoio profissional se necessário.



--- Fim do Script ---
